In [13]:
#Import the Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
!pip install wordcloud
!pip install textblob
!python -m spacy download en

[nltk_data] Downloading package stopwords to C:\Users\ASUS TUFF
[nltk_data]     GAMING\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use
the full pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [15]:
#Scrape the Data
#import pandas as pd
#from google_play_scraper import Sort, reviews_all

#us_reviews = reviews_all(
#   'com.alodokter.android',
#    sleep_milliseconds=0,
#    lang='en',
#    country='us',
#    sort=Sort.NEWEST
#)

#df_busu = pd.DataFrame(us_reviews)
#df_busu.to_csv('alodokter_reviews.csv', index=False)

#print("Reviews have been saved to 'reviews.csv'.")
#Apply this to several other telemedicine startup reviews.

ModuleNotFoundError: No module named 'google_play_scraper'

In [16]:
#Create the Model
#importing the training data
imdb_data=pd.read_csv('C:\\Users\\ASUS TUFF GAMING\\Downloads\\indonlu\\IMDB Dataset.csv')
print(imdb_data.shape)
print(imdb_data.head())

#Summary of the dataset
print(imdb_data.describe())

(50000, 2)
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
                                                   review sentiment
count                                               50000     50000
unique                                              49582         2
top     Loved today's show!!! It was a variety and not...  positive
freq                                                    5     25000


In [17]:
#Change it to Binary
imdb_data['sentiment'] = imdb_data['sentiment'].replace('positive',1)
imdb_data['sentiment'] = imdb_data['sentiment'].replace('negative',0)
print(imdb_data.head())

                                              review  sentiment
0  One of the other reviewers has mentioned that ...          1
1  A wonderful little production. <br /><br />The...          1
2  I thought this was a wonderful way to spend ti...          1
3  Basically there's a family where a little boy ...          0
4  Petter Mattei's "Love in the Time of Money" is...          1


In [18]:
#split the dataset
#train dataset
train_reviews=imdb_data.review[:40000]
train_sentiments=imdb_data.sentiment[:40000]
#test dataset
test_reviews=imdb_data.review[40000:]
test_sentiments=imdb_data.sentiment[40000:]
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)

(40000,) (40000,)
(10000,) (10000,)


In [19]:
#Data Preprocessing
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(denoise_text)

In [20]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_special_characters)

#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(simple_stemmer)

In [21]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_stopwords)

{'having', "shan't", "mustn't", "mightn't", 'very', 've', 'all', 'wasn', 'with', 'or', 'from', 'm', "doesn't", 'isn', 'd', 'under', 'have', 'those', 'how', 'shouldn', 'yourself', 'ain', 'for', "needn't", "it's", 'up', 'couldn', 'while', 'through', 'you', 'is', 'few', "wasn't", 'has', 'it', 'i', 'because', 're', 'wouldn', 'were', 'into', 'at', "you'd", 'nor', 'her', 'his', 'my', 'off', 'am', 'we', 'no', 'below', 'themselves', "you'll", "should've", 'out', "that'll", 'during', 'them', 'too', 'o', "she's", 'only', 'herself', 'each', 'ma', 'above', 'just', "isn't", 'what', 'down', 'she', "won't", 'the', "don't", 'than', 'in', "hasn't", 'why', 'ours', "hadn't", 'needn', 'had', "couldn't", 'own', 'if', 'itself', 'weren', 'over', 'he', 'there', 'this', 'been', 'yourselves', 'when', 'before', 'do', 'now', 'are', 'once', 'about', 'ourselves', 'a', 'on', 'these', 'don', "aren't", 'as', 'then', 'same', 'of', 'any', 'so', 'both', 'myself', 'doing', 'again', 'further', 'll', 'doesn', "you've", 'mos

In [22]:
#Count vectorizer for bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train_reviews=cv.fit_transform(train_reviews)
#transformed test reviews
cv_test_reviews=cv.transform(test_reviews)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)

BOW_cv_train: (40000, 6183315)
BOW_cv_test: (10000, 6183315)


In [23]:
#Tfidf vectorizer
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(train_reviews)
#transformed test reviews
tv_test_reviews=tv.transform(test_reviews)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

Tfidf_train: (40000, 6183315)
Tfidf_test: (10000, 6183315)


In [24]:
#training the model
mnb=MultinomialNB()

In [25]:
# Reshape the train_sentiments array to be 1-dimensional
train_sentiments = np.ravel(train_sentiments)

# Continue with the model fitting and evaluation steps
mnb_bow = mnb.fit(cv_train_reviews, train_sentiments)

#fitting the svm for bag of words
mnb_bow=mnb.fit(cv_train_reviews,train_sentiments)
print(mnb_bow)
#fitting the svm for tfidf features
mnb_tfidf=mnb.fit(tv_train_reviews,train_sentiments)
print(mnb_tfidf)

#Predicting the model for bag of words
mnb_bow_predict=mnb.predict(cv_test_reviews)
print(mnb_bow_predict)
#Predicting the model for tfidf features
mnb_tfidf_predict=mnb.predict(tv_test_reviews)
print(mnb_tfidf_predict)

MultinomialNB()
MultinomialNB()
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


In [26]:
#Accuracy score for bag of words
mnb_bow_score=accuracy_score(test_sentiments,mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)
#Accuracy score for tfidf features
mnb_tfidf_score=accuracy_score(test_sentiments,mnb_tfidf_predict)
print("mnb_tfidf_score :",mnb_tfidf_score)

mnb_bow_score : 0.7682
mnb_tfidf_score : 0.7677


In [27]:
#Classification report for bag of words
mnb_bow_report=classification_report(test_sentiments,mnb_bow_predict,target_names=['Positive','Negative'])
print(mnb_bow_report)
#Classification report for tfidf features
mnb_tfidf_report=classification_report(test_sentiments,mnb_tfidf_predict,target_names=['Positive','Negative'])
print(mnb_tfidf_report)

              precision    recall  f1-score   support

    Positive       0.76      0.78      0.77      4993
    Negative       0.77      0.76      0.77      5007

    accuracy                           0.77     10000
   macro avg       0.77      0.77      0.77     10000
weighted avg       0.77      0.77      0.77     10000

              precision    recall  f1-score   support

    Positive       0.76      0.78      0.77      4993
    Negative       0.77      0.76      0.77      5007

    accuracy                           0.77     10000
   macro avg       0.77      0.77      0.77     10000
weighted avg       0.77      0.77      0.77     10000



In [28]:
#confusion matrix for bag of words
cm_bow=confusion_matrix(test_sentiments,mnb_bow_predict,labels=[1,0])
print(cm_bow)
#confusion matrix for tfidf features
cm_tfidf=confusion_matrix(test_sentiments,mnb_tfidf_predict,labels=[1,0])
print(cm_tfidf)

[[3803 1204]
 [1114 3879]]
[[3791 1216]
 [1107 3886]]


In [29]:
#Translate the Reviews
!pip install deep_translator
from deep_translator import GoogleTranslator
# Import the CSV
medcall_reviews = pd.read_csv("C:\\Users\\ASUS TUFF GAMING\\PycharmProjects\\pythonProject4\\medicall_reviews.csv")

# Define the column to translate
column_to_translate = 'content'

# Translate the column
medcall_reviews['content'] = medcall_reviews[column_to_translate].apply(lambda x: GoogleTranslator(source='auto', target='en').translate(x))

In [30]:
# Data Preparation
# Drop Unimportant Data
columns_to_drop = ['reviewId', 'userImage', 'thumbsUpCount', 'reviewCreatedVersion', 'replyContent', 'repliedAt','appVersion']
medcall_reviews = medcall_reviews.drop(columns_to_drop, axis=1)

In [31]:
#Download more Libraries!
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.tokenize import word_tokenize
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to C:\Users\ASUS TUFF
[nltk_data]     GAMING\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [32]:
# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

In [33]:
def analyze_sentiment_swn(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Perform POS tagging
    pos_tags = pos_tag(tokens)

    # Map Penn Treebank POS tags to WordNet POS tags
    tag_map = {
        'NN': 'n',
        'VB': 'v',
        'JJ': 'a',
        'RB': 'r'
    }

    # Analyze sentiment for each token
    scores = []
    for token, pos in pos_tags:
        # Get the WordNet POS tag
        wordnet_pos = tag_map.get(pos[:2], 'n')

        # Lemmatize the token
        lemma = lemmatizer.lemmatize(token, pos=wordnet_pos)

        # Get the synsets for the lemma and POS tag
        synsets = list(swn.senti_synsets(lemma, pos=wordnet_pos))

        # Calculate the sentiment score
        if synsets:
            synset = synsets[0]
            senti_score = synset.pos_score() - synset.neg_score()
            scores.append(senti_score)

    # Return the average sentiment score
    if scores:
        return sum(scores) / len(scores)
    else:
        return 0.0

In [39]:
from nltk.sentiment import SentimentIntensityAnalyzer

def analyze_sentiment_vader(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(text)
    compound_score = sentiment_scores['compound']
    return compound_score

# Apply the Model to Reviews
def analyze_sentiment(text):
    sentiment_score = mnb.predict(text)
    return sentiment_score

#Apply the Model to Reviews
def analyze_sentiment_MNB(text):
    text_vector = cv.transform([text])  # Transform the text using the CountVectorizer
    sentiment_score = mnb.predict(text_vector)  # Predict the sentiment using the MultinomialNB classifier
    return sentiment_score[0]

In [44]:
# Apply Lexicon Sentiment Analysis
medcall_reviews['SentiWordNet_Score'] = medcall_reviews['content'].apply(analyze_sentiment_swn)
medcall_reviews['VADER_Score'] = medcall_reviews['content'].apply(analyze_sentiment_vader)
medcall_reviews['MNB Score'] = medcall_reviews['content'].apply(analyze_sentiment_MNB)

medcall_reviews.to_csv(r'C:\Users\ASUS TUFF GAMING\Downloads\new_medcall.csv', index=False)
print("Hooray!")

Hooray!


In [ ]:
#Apply it to Alodokter
alodokter_reviews = pd.read_csv('C:\\Users\\ASUS TUFF GAMING\\PycharmProjects\\pythonProject4\\alodokter_reviews.csv')
alodokter_reviews = alodokter_reviews.drop(columns_to_drop, axis=1)
# Translate the column
alodokter_reviews['content'] = alodokter_reviews[column_to_translate].apply(lambda x: GoogleTranslator(source='auto', target='en').translate(x))
alodokter_reviews.head()

In [ ]:
# Apply Lexicon Sentiment Analysis
alodokter_reviews['SentiWordNet_Score'] = alodokter_reviews['content'].apply(analyze_sentiment_swn)
alodokter_reviews['VADER_Score'] = alodokter_reviews['content'].apply(analyze_sentiment_vader)
alodokter_reviews['MNB Score'] = alodokter_reviews['content'].apply(analyze_sentiment_MNB)

alodokter_reviews.to_csv(r'C:\Users\ASUS TUFF GAMING\Downloads\new_alodokter.csv', index=False)
print("Hooray!")

In [ ]:
#Apply it to Halodoc
halodoc_reviews = pd.read_csv('C:\\Users\\ASUS TUFF GAMING\\PycharmProjects\\pythonProject4\\halodoc_reviews.csv')
halodoc_reviews = halodoc_reviews.drop(columns_to_drop, axis=1)
# Translate the column
halodoc_reviews['content'] = halodoc_reviews[column_to_translate].apply(lambda x: GoogleTranslator(source='auto', target='en').translate(x))
halodoc_reviews.head()

In [ ]:
# Apply Lexicon Sentiment Analysis
halodoc_reviews['SentiWordNet_Score'] = halodoc_reviews['content'].apply(analyze_sentiment_swn)
halodoc_reviews['VADER_Score'] = halodoc_reviews['content'].apply(analyze_sentiment_vader)
halodoc_reviews['MNB Score'] = halodoc_reviews['content'].apply(analyze_sentiment_MNB)

halodoc_reviews.to_csv(r'C:\Users\ASUS TUFF GAMING\Downloads\new_halodoc.csv', index=False)
print("Hooray!")

In [ ]:
#Apply it to Klikdokter
klikdokter_reviews = pd.read_csv('C:\\Users\\ASUS TUFF GAMING\\PycharmProjects\\pythonProject4\\klikdokter_reviews.csv')
klikdokter_reviews = klikdokter_reviews.drop(columns_to_drop, axis=1)
# Translate the column
klikdokter_reviews['content'] = klikdokter_reviews[column_to_translate].apply(lambda x: GoogleTranslator(source='auto', target='en').translate(x))
klikdokter_reviews.head()

In [ ]:
# Apply Lexicon Sentiment Analysis
klikdokter_reviews['SentiWordNet_Score'] = klikdokter_reviews['content'].apply(analyze_sentiment_swn)
klikdokter_reviews['VADER_Score'] = klikdokter_reviews['content'].apply(analyze_sentiment_vader)
klikdokter_reviews['MNB Score'] = klikdokter_reviews['content'].apply(analyze_sentiment_MNB)

klikdokter_reviews.to_csv(r'C:\Users\ASUS TUFF GAMING\Downloads\new_klikdokter.csv', index=False)
print("Hooray!")

In [ ]:
#Apply it to Good Doctor
thegooddoctor_reviews = pd.read_csv('C:\\Users\\ASUS TUFF GAMING\\PycharmProjects\\pythonProject4\\thegooddoctor_reviews.csv')
thegooddoctor_reviews = thegooddoctor_reviews.drop(columns_to_drop, axis=1)
# Translate the column
thegooddoctor_reviews['content'] = thegooddoctor_reviews[column_to_translate].apply(lambda x: GoogleTranslator(source='auto', target='en').translate(x))
thegooddoctor_reviews.head()

In [ ]:
# Apply Lexicon Sentiment Analysis
thegooddoctor_reviews['SentiWordNet_Score'] = thegooddoctor_reviews['content'].apply(analyze_sentiment_swn)
thegooddoctor_reviews['VADER_Score'] = thegooddoctor_reviews['content'].apply(analyze_sentiment_vader)
thegooddoctor_reviews['MNB Score'] = thegooddoctor_reviews['content'].apply(analyze_sentiment_MNB)

thegooddoctor_reviews.to_csv(r'C:\Users\ASUS TUFF GAMING\Downloads\new_thegooddoctor.csv', index=False)
print("Hooray!")

In [ ]:
#Apply it to Satu Sehat
SATUSEHAT_reviews = pd.read_csv('C:\\Users\\ASUS TUFF GAMING\\PycharmProjects\\pythonProject4\\SATUSEHAT_reviews.csv')
SATUSEHAT_reviews = SATUSEHAT_reviews.drop(columns_to_drop, axis=1)
# Translate the column
SATUSEHAT_reviews['content'] = SATUSEHAT_reviews[column_to_translate].apply(lambda x: GoogleTranslator(source='auto', target='en').translate(x))
SATUSEHAT_reviews.head()

In [ ]:
# Apply Lexicon Sentiment Analysis
SATUSEHAT_reviews['SentiWordNet_Score'] = thegooddoctor_reviews['content'].apply(analyze_sentiment_swn)
SATUSEHAT_reviews['VADER_Score'] = thegooddoctor_reviews['content'].apply(analyze_sentiment_vader)
SATUSEHAT_reviews['MNB Score'] = thegooddoctor_reviews['content'].apply(analyze_sentiment_MNB)

SATUSEHAT_reviews.to_csv(r'C:\Users\ASUS TUFF GAMING\Downloads\new_SATUSEHAT.csv', index=False)
print("Hooray!")